In [244]:
! pip install python-dotenv
! pip install pymorphy2 nltk scikit-learn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.5/55.5 kB 802.0 kB/s eta 0:00:000:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 1.1 MB/s eta 0:00:0000:0100:01


In [253]:
import pandas as pd
import numpy as np
import gdown
import os
import json
from dotenv import load_dotenv
import pymorphy2
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import nltk
import ast

nltk.download('punkt')
nltk.download('stopwords')

from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import Pipeline, FunctionTransformer
from sklearn.feature_extraction.text import TfidfVectorizer

[nltk_data] Downloading package punkt to /home/kitsu/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /home/kitsu/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [213]:
ROOT_DIR: str = os.path.dirname(os.path.abspath(os.curdir))
DATASET_PATH: str = f"{ROOT_DIR}/source_data/superset_hr.xlsx"
SKILLS_JSON_PATH: str = f"{ROOT_DIR}/source_data/skills_dictionary.json"
GOOGLE_COLAB_PATH: str = '/content/superset_hr.xlsx'

load_dotenv()
SKILL_DICT_URL = os.environ.get('SKILL_DICT_URL')
DATASET_URL = os.environ.get('DATASET_URL')

if DATASET_URL is None:
    print('>>> .env was not found!', end='\n\n')
    DATASET_URL = input('>>> Provide DATASET url for downloading: ')

    if SKILL_DICT_URL is None:
        SKILL_DICT_URL = input('>>> Provide JSON url for downloading: ')

In [214]:
def get_skill_dict() -> dict[str: list[int]]:
    try:
        with open(file=SKILLS_JSON_PATH, mode='r') as input_json_file:
            data = json.load(input_json_file)
    except FileNotFoundError:
        print(f"Can't open file from path: {SKILLS_JSON_PATH}", end='\n\n')
        file_name = gdown.download(SKILL_DICT_URL, fuzzy=True)
        os.makedirs(os.path.dirname(SKILLS_JSON_PATH), exist_ok=True)
        os.rename(os.path.abspath(file_name), SKILLS_JSON_PATH)
        print(f'Moved to: {SKILLS_JSON_PATH}', end='\n\n')
        
        with open(file=SKILLS_JSON_PATH, mode='r') as input_json_file:
            data = json.load(input_json_file)
    return data

def get_original_dataframe() -> pd.DataFrame:
    try:
        df = pd.read_excel(DATASET_PATH)
    except FileNotFoundError:
        try:
            print(f"Can't open file from path: {DATASET_PATH}", end='\n\n')
            df = pd.read_excel(GOOGLE_COLAB_PATH)
        except FileNotFoundError:
            print(f"Can't open file from path: {GOOGLE_COLAB_PATH}", end='\n\n')
            file_name = gdown.download(DATASET_URL, fuzzy=True)
            os.makedirs(os.path.dirname(DATASET_PATH), exist_ok=True)
            os.rename(os.path.abspath(file_name), DATASET_PATH)
            print(f'Moved to: {DATASET_PATH}', end='\n\n')
            df = pd.read_excel(DATASET_PATH)
    finally:
        print('Success!')
        return df

In [215]:
skills_dict = get_skill_dict()

Can't open file from path: /home/kitsu/ML_HR/source_data/skills_dictionary.json



Downloading...
From: https://drive.google.com/uc?id=1fZf4B01VUKrFCDo1PmI5waUWkLZANDBT
To: /home/kitsu/ML_HR/Notebooks/skills_dictionary.json
100%|██████████| 458k/458k [00:00<00:00, 1.46MB/s]

Moved to: /home/kitsu/ML_HR/source_data/skills_dictionary.json



In [4]:
df_original = get_original_dataframe()

Success!


In [5]:
print(df_original.shape)
df_original.sample(3)

(175455, 24)


,Unnamed: 0.2,Unnamed: 0.1,Unnamed: 0,id,income_name,name,published_at,created_at,collected_at,url,...,address__city,schedule__name,grade,employment__name,key_skills__names,languages__names,exchange_rate,salary_from_gross,salary_to_gross,salary_average
108996,110841,110841,110840,94616767,1C оператор,Оператор 1С,2024-03-27T15:38:03+0300,2024-03-27T15:38:03+0300,2024-03-29 13:42:32.044,https://api.hh.ru/vacancies/94616767?host=hh.ru,...,Уральск,Полный день,Junior (1-3),Полная занятость,"['1С: Предприятие 8', 'Складской Учет', 'Докум...",[],0.206,32589.2,NaN,NaN
1841,1873,1873,1873,88647327,DevOps,DevOps инженер,2024-01-19T18:46:03+0300,2024-01-19T18:46:03+0300,2024-01-27 00:00:00.000,https://api.hh.ru/vacancies/88647327?host=hh.ru,...,Санкт-Петербург,Полный день,Junior (1-3),Полная занятость,"['Prometheus', 'Grafana', 'Docker', 'Kubernete...",[],NaN,NaN,NaN,NaN
162034,165150,165150,165152,97144139,Product manager,Product Marketing Manager,2024-04-16T17:25:04+0300,2024-04-16T17:25:04+0300,2024-04-17 10:50:23.602,https://api.hh.ru/vacancies/97144139?host=hh.ru,...,NaN,Удаленная работа,Middle (3-6),Полная занятость,"['Product Marketing', 'Маркетинговые коммуника...",[],1.000,150000.0,NaN,NaN


In [317]:
def print_df_info(df: pd.DataFrame) -> None:
    print(f"Shape: {df.shape}")
    print('-' * 50)
    display(df.sample(3))
    print('-' * 50)
    display(df.info())
    print('-' * 50)
    display(df.isna().sum())
    print('-' * 50)


def get_clear_df_version(df: pd.DataFrame) -> pd.DataFrame:
    df_new = df[['income_name', 'area__name', 'schedule__name', 'grade',
                 'key_skills__names', 'salary__currency',
                 'exchange_rate', 'salary_from_gross', 'salary_to_gross', 'salary_average']].copy()

    df_new = df_new.dropna(subset=['salary_from_gross', 'salary_to_gross', 'salary_average'], how='all')
    df_new = df_new.dropna(subset=['income_name'])

    df_new['not_rur'] = df_new['exchange_rate'] != 1
    df_new['not_rur'] = df_new['not_rur'].astype(int)
    df_new = df_new.drop(['salary__currency', 'exchange_rate'], axis=1)

    return df_new


def get_difference_percentiles(df: pd.DataFrame) -> tuple[float, float, float]:
    filtered_df = df.dropna(subset=['salary_from_gross', 'salary_to_gross']).copy()
    filtered_df.loc[:, 'difference'] = filtered_df['salary_to_gross'] - filtered_df['salary_from_gross']
    condition = filtered_df['difference'] > 0.8 * filtered_df['salary_to_gross']
    filtered_df.loc[condition, ['salary_from_gross', 'difference']] = np.nan

    filtered_df = filtered_df.dropna(subset=['salary_from_gross', 'salary_to_gross'])

    filtered_df = filtered_df[filtered_df['salary_from_gross'] >= 1000]
    filtered_df = filtered_df[filtered_df['salary_to_gross'] >= 10000]

    filtered_df.loc[:, 'difference_ratio'] = np.where(filtered_df['salary_to_gross'] != 0, 
                                                      filtered_df['difference'] / filtered_df['salary_to_gross'], 
                                                      np.nan)

    perc25 = filtered_df['difference_ratio'].quantile(0.25)
    perc50 = filtered_df['difference_ratio'].quantile(0.50)
    perc75 = filtered_df['difference_ratio'].quantile(0.75)

    return perc25, perc50, perc75


def fill_na_salary(df: pd.DataFrame, coef) -> pd.DataFrame:
    result_df = df.copy()
    result_df['salary_to_gross'] = result_df['salary_to_gross'].fillna(result_df['salary_from_gross'] / (1-coef))
    result_df['salary_from_gross'] = result_df['salary_from_gross'].fillna(result_df['salary_to_gross'] * (1-coef))
    result_df['salary_average'] = result_df['salary_average'].fillna((result_df['salary_to_gross'] + result_df['salary_from_gross']) / 2)
    
    result_df[['salary_to_gross', 'salary_from_gross', 'salary_average']] = result_df[['salary_to_gross', 'salary_from_gross', 'salary_average']].astype(float)
    return result_df


def exctract_features(df: pd.DataFrame) -> pd.DataFrame:
    transformers = [
        ('schedule_name', OneHotEncoder(sparse_output=False, drop='first'), ['schedule__name']),
        ('grade', OneHotEncoder(sparse_output=False, drop='first'), ['grade']),
        ('income_name', OneHotEncoder(sparse_output=False, drop='first'), ['income_name'])
    ]

    pipeline = Pipeline(steps=[
        ('column_transformer', ColumnTransformer(transformers=transformers, remainder='passthrough'))
    ])

    display(pipeline)

    df_transformed = pipeline.fit_transform(df)
    column_names = pipeline.named_steps['column_transformer'].get_feature_names_out()
    transofrmed_cols = [x for x in column_names if 'remainder' not in x]
    column_names = [x.replace('remainder__', '') if x not in transofrmed_cols else x for x in column_names]
    
    df_new = pd.DataFrame(df_transformed, columns=column_names)
    df_new[transofrmed_cols] = df_new[transofrmed_cols].astype(float)
    
    return df_new


def convert_skills(skill_string: str, skills_dict: dict) -> tuple[list[str], int, list[str]]:
    try:
        skill_list = eval(skill_string)
        if not isinstance(skill_list, list):
            return np.nan, 0, []
    except:
        return np.nan, 0, []

    converted_skills = []
    unconverted_skills = []

    for skill in skill_list:
        found = False
        for main_skill, variations in skills_dict.items():
            if skill in variations:
                converted_skills.append(main_skill)
                found = True
                break
        if not found:
            unconverted_skills.append(skill)

    return converted_skills, unconverted_skills


def preprocess_key_skills(df: pd.DataFrame, s_dict) -> pd.DataFrame:
    df_new = df.copy()
    df_new[['skills', 'unconverted_skills']] = df_new['key_skills__names'].apply(
    lambda x: pd.Series(convert_skills(x, s_dict)))
    df_new = df_new.drop(['key_skills__names'], axis=1)

    return df_new


morph = pymorphy2.MorphAnalyzer()
nltk_stop_words  = set(stopwords.words('russian'))
custom_words = {',', ':', 'работа'}
stop_words = nltk_stop_words.union(custom_words)


def tokenize_and_lemmatize(skill_list: list[str]) -> list[str]:
    tokens = []
    for skill in skill_list:
        words = word_tokenize(skill)
        lemmas = [morph.parse(word.lower())[0].normal_form for word in words if word.lower() not in stop_words]
        tokens.extend(lemmas)
    return tokens


def process_skills(df: pd.DataFrame, freq_cutoff: int = 100) -> pd.DataFrame:
    df_processed = df.copy()

    df_processed['unskills_processed'] = df_processed['unconverted_skills'].apply(tokenize_and_lemmatize)
    unskills_counts = df_processed['unskills_processed'].explode().value_counts()
    frequent_skills = unskills_counts[unskills_counts >= freq_cutoff].index

    df_processed['unskills_filtered'] = df_processed['unskills_processed'].apply(lambda skills: [skill for skill in skills if skill in frequent_skills])
    df_processed['skills_plus'] = df_processed['skills'] + df_processed['unskills_filtered']
    df_processed = df_processed[df_processed['skills_plus'].apply(lambda x: len(x) > 0)]

    df_processed = df_processed.drop(['unconverted_skills', 
                                      'skills', 
                                      'unskills_processed', 
                                      'unskills_filtered'], axis = 1)
    return df_processed

def vectorize_skills(df: pd.DataFrame) -> pd.DataFrame:
    df_vect = df.copy()
    df_vect['skills_plus'] = df_vect['skills_plus'].apply(lambda x: ' '.join(x))

    vectorizer = TfidfVectorizer()
    display(vectorizer)
    skills_tfidf = vectorizer.fit_transform(df_vect['skills_plus'])
    skills_df = pd.DataFrame(skills_tfidf.toarray(), columns=vectorizer.get_feature_names_out())

    df_vect = df_vect.reset_index()
    df_merged = pd.concat([df_vect, skills_df], axis=1)

    df_merged = df_merged.drop(['skills_plus'], axis=1)
    df_merged[['not_rur',
               'salary_from_gross', 
               'salary_to_gross', 
               'salary_average']] = df_merged[['not_rur',
                                               'salary_from_gross', 
                                               'salary_to_gross', 
                                               'salary_average']].astype(float)

    return df_merged

In [318]:
pipeline_preprocess = Pipeline(steps=[
    ('clear_df', FunctionTransformer(get_clear_df_version)),

    ('fill_na_salary', FunctionTransformer(
        func=lambda df: fill_na_salary(df, get_difference_percentiles(df)[0]),
        validate=False,
    )),

    ('extract_features', FunctionTransformer(exctract_features, validate=False)),

    ('preprocess_key_skills', FunctionTransformer(
        func=lambda df: preprocess_key_skills(df, skills_dict),
        validate=False,
    )),
    
    ('process_skills', FunctionTransformer(
        func=lambda df: process_skills(df, 100),
        validate=False,
    )),
    
    ('vectorize_skills', FunctionTransformer(vectorize_skills, validate=False))
    ], 
    verbose=True).set_output(transform="pandas")

display(pipeline_preprocess)

Pipeline(steps=[('clear_df',
                 FunctionTransformer(func=<function get_clear_df_version at 0x7f2020c0d000>)),
                ('fill_na_salary',
                 FunctionTransformer(func=<function <lambda> at 0x7f20a4663d00>)),
                ('extract_features',
                 FunctionTransformer(func=<function exctract_features at 0x7f200cf1d900>)),
                ('preprocess_key_skills',
                 FunctionTransformer(func=<function <lambda> at 0x7f200a153490>)),
                ('process_skills',
                 FunctionTransformer(func=<function <lambda> at 0x7f200a151f30>)),
                ('vectorize_skills',
                 FunctionTransformer(func=<function vectorize_skills at 0x7f200cf1e8c0>))],
         verbose=True)

In [319]:
df_after_preprocess: pd.DataFrame = pipeline_preprocess.fit_transform(df_original)

[Pipeline] .......... (step 1 of 6) Processing clear_df, total=   0.1s
[Pipeline] .... (step 2 of 6) Processing fill_na_salary, total=   0.0s


Pipeline(steps=[('column_transformer',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('schedule_name',
                                                  OneHotEncoder(drop='first',
                                                                sparse_output=False),
                                                  ['schedule__name']),
                                                 ('grade',
                                                  OneHotEncoder(drop='first',
                                                                sparse_output=False),
                                                  ['grade']),
                                                 ('income_name',
                                                  OneHotEncoder(drop='first',
                                                                sparse_output=False),
                                                  ['income_name'])]))])

[Pipeline] .. (step 3 of 6) Processing extract_features, total=   1.9s
[Pipeline]  (step 4 of 6) Processing preprocess_key_skills, total=  22.4s
[Pipeline] .... (step 5 of 6) Processing process_skills, total=  32.9s


TfidfVectorizer()

[Pipeline] .. (step 6 of 6) Processing vectorize_skills, total=   2.1s


In [320]:
print_df_info(df_after_preprocess)

Shape: (77828, 869)
--------------------------------------------------


,index,schedule_name__schedule__name_Гибкий график,schedule_name__schedule__name_Полный день,schedule_name__schedule__name_Сменный график,schedule_name__schedule__name_Удаленная работа,grade__grade_Junior (1-3),grade__grade_Middle (3-6),grade__grade_Senior (>6),income_name__income_name_1C оператор,income_name__income_name_1С администратор,...,эксплуатационные,электронный,энергичность,этика,этикет,эффективностью,юридический,юридическими,язык,яндекс
48146,67379,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
48653,68130,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
43252,60166,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


--------------------------------------------------
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 77828 entries, 0 to 77827
Columns: 869 entries, index to яндекс
dtypes: float64(867), int64(1), object(1)
memory usage: 516.0+ MB


None

--------------------------------------------------


index                                             0
schedule_name__schedule__name_Гибкий график       0
schedule_name__schedule__name_Полный день         0
schedule_name__schedule__name_Сменный график      0
schedule_name__schedule__name_Удаленная работа    0
                                                 ..
эффективностью                                    0
юридический                                       0
юридическими                                      0
язык                                              0
яндекс                                            0
Length: 869, dtype: int64

--------------------------------------------------


In [321]:
print(df_after_preprocess.select_dtypes(include='object').columns)
df_after_preprocess.select_dtypes(include='object').head(3)

Index(['area__name'], dtype='object')


,area__name
0,Санкт-Петербург
1,Санкт-Петербург
2,Санкт-Петербург


In [322]:
df_after_preprocess.to_csv('hh_dataset.csv', index=False)